# Import and Map Business Terms to Data Headers

## Introduction

This notebook imports a business glossary for the industry accelerator into Watson Knowledge Catalog (WKC) for use in governing data assets.

The notebook category and business terms are imported and then business terms are published into Watson Knowledge Catalog. The category and business terms csvs files are included with the project.

This notebook is optional. The remaining project runs as expected even if this notebook is not used. 


#### Create Catalog

The catalog must be manually created. Under **Catalogs** in the navigation menu, select **All Catalogs** and select **New Catalog**. Enter the name for the catalog and the description if necessary and create the catalog. If the user has already created the catalog this step can be skipped and the existing catalog name should be specified in the code cell below.

 **catalog_name :** Name of the catalog that we would like to publish the csv to. This catalog is created based on the instructions above or an existing catalog.

In [1]:
# imports for the rest APIs interactions with WKC
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from pandas.io.json import json_normalize
import pandas as pd
import os
import time 

s=requests.session()

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [2]:
catalog_name = 'Industry Accelerators'

## User Inputs
1. **wkcURL**: The base url used to call the apis of data platform.
2. **categories_csv**: Name of the csv file containing the categories.
3. **terms_csv**: Name of the csv file containing the business terms and their definitions.
4. **category_name :** Name of the business term category corresponding to the project.

The user does not need to change the code cell below, unless they run this notebook on a different environment or changed the name of the csv file with categories and business terms.


In [3]:
wkcURL = os.environ['RUNTIME_ENV_APSX_URL']
categories_csv="reducing-severe-weather-risk-in-insurance-glossary-categories.csv"
terms_csv="reducing-severe-weather-risk-in-insurance-glossary-terms.csv"
category_name = "Reducing Severe Weather Risk in Insurance"

## 1. Import the Categories and Business terms 
If the user is successfully authenticated, then we complete the following steps 
1. Read the business categories file and import the category `Reducing Severe Weather Risk in Insurance` under `Industry Accelerators`
2. Read the business terms file and import the business terms with definitions and related/part of terms for `Reducing Severe Weather Risk in Insurance`.
3. The imported business terms will be saved as a draft, publish the business terms programmatically. 

#### Import the Category

Below cell reads catgories csv file and import the category `Reducing Severe Weather Risk in Insurance` under `Industry Accelerators`.  

In [4]:

wslib.download_file(categories_csv)

# Create a header
importheaders = {
    'accept': 'application/json',
    'Authorization': 'Bearer '+os.environ['USER_ACCESS_TOKEN']  
    #'Content-Type': 'multipart/form-data'
}

# Specify merge option 
params = (
    ('merge_option', 'all'),
)

# remove the index if necessary
time.sleep(10)

# Specify the file name to import
files = {
    'file': (categories_csv, open(categories_csv, 'rb')),
}

try:
    categoryimport_url = wkcURL+'/v3/governance_artifact_types/category/import'
    cat_import = s.post(categoryimport_url, headers=importheaders, params=params, files=files)
    if cat_import.status_code in (200,202):
        print("Category import has",json.loads(cat_import.text)['status'])
    else:
        print("Category import is unsuccessful,", json.loads(cat_import.text)['errors'][0]['message'])
except:
    print("The below error has occurred. Please ensure that categories csv file exists")
    raise



Category import has SUCCEEDED


#### Import Business terms

Read the business terms file and import the business terms with definitions and related/part of terms for `Reducing Severe Weather Risk in Insurance`. 


In [5]:

wslib.download_file(terms_csv)



files = {
    'file': (terms_csv, open(terms_csv, 'rb')),
}

try:
    termsimport_url =wkcURL+'/v3/governance_artifact_types/glossary_term/import'
    term_import = s.post(termsimport_url, headers=importheaders, params=params, files=files)
    if term_import.status_code in (200,202):
        wf_json=json.loads(term_import.text)
        print("Terms import has",wf_json['status'])
        print("Number of Business terms imported:",max(json.loads(term_import.text)['operations_count']['glossary_term']['IMPORT_MODIFY'],json.loads(term_import.text)['operations_count']['glossary_term']['IMPORT_CREATE']))
    else:
        print("Import of Business Terms failed.")
        raise Exception(json.loads(term_import.text)['errors'][0]['message'])
except:
    print("The below error has occurred. Please check your WKC plan and Business Terms csvs")
    print(json.loads(term_import.text)['messages']['resources'])
    raise



Terms import has SUCCEEDED
Number of Business terms imported: 12


#### Fetch imported terms draft and publish the terms

If the business terms are successfully imported in the previous cell, the imported business terms have been saved in draft status. The below cell publishes the business terms.

In [6]:
headers = {
'Content-Type': "application/json",
'Authorization': "Bearer "+os.environ['USER_ACCESS_TOKEN']

}
try:
    wf_url=wkcURL+"/v3/workflows/"+wf_json['workflow_id']+"?includeUserTasks=true"
    wf_response=s.get(wf_url,headers=headers)
    if wf_response.status_code in (200,202):
        idtopublish=json.loads(wf_response.text)['entity']['user_tasks'][0]['metadata']['task_id']
except:
    print("The below error has occurred. Please ensure that terms are imported correctly")
    raise    
    
    
## Publish the Business terms
data = {
    "action": "complete",
    "form_properties": [
        {
            "id": "action",
            "value": "#publish"
        }
    ]
}
publish_url=wkcURL+'/v3/workflow_user_tasks/'+idtopublish+'/actions'


try:
    publish = s.post(publish_url, headers=headers, json=data, verify=False)

    if publish.status_code in (200,202,204):
        print("The Business terms are published.")
except:
    print("The below error has occurred. Please ensure that terms are imported correctly")
    raise
    


The Business terms are published.


When the Business terms are published, navigate to **Governance -> Categories -> Industry Accelerators** to find the category and business terms pertaining to this accelerator.<br>

In [7]:
s.close()

<hr>
Sample Materials, provided under <a href="https://github.com/IBM/Industry-Accelerators/blob/master/CPD%20SaaS/LICENSE" target="_blank" rel="noopener noreferrer">license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020, 2023. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>